In [ ]:
pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  --upgrade pip

In [ ]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  \
pinecone-client \
pandas \
langchain \
sentence-transformers

In [ ]:
from pinecone import Pinecone
import os
import pandas as pd
import numpy as np

#from sentence_transformers import SentenceTransformer
from langchain.document_loaders import TextLoader
from langchain.document_loaders import  DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

# Prepare Data - New Articles

In [ ]:
# load multiple document and process documants
loader = DirectoryLoader("./bbc/sport", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
# split the text into smaler chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_spliter.split_documents(documents)

In [ ]:
len(texts)

# Create Embeddings and Vectors

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
os.environ["CURL_CA_BUNDLE"] = ""

In [ ]:
# average_word_embeddings_komninos
# dimension = 300
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
import requests

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

In [ ]:
def model(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}}, verify=False)
    return response.json()

In [ ]:
# Must be a list
news_text = []
for text in texts:
    news_text.append(text.page_content)
news_text[0]

In [ ]:
# model = SentenceTransformer('average_word_embeddings_komninos')
# encoded_news = model(news_text, show_progress_bar=True)
encoded_news = model(news_text)

In [ ]:
len(encoded_news[0])

# Setupup Pinecone index
	- Make sure the dimenionality of the embeddings matches the dimensionality of the index

In [ ]:
pc.list_indexes()

In [ ]:
pc.delete_index('prices')

In [ ]:
pc.list_indexes()

In [ ]:
from pinecone import PodSpec
pc.create_index(
	'news', 
	dimension=300 ,
	metric='cosine',
	spec=PodSpec(
        	environment="gcp-starter",
        	pod_type="starter",
        	pods=1,
        	replicas=1,
        	shards=1
	)
)

# Upsert vector data

# Query vector data

# Query with metadata filters

# Cleanup